<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Compare_all_bigdata_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datarevenue.com/en-blog/pandas-vs-dask-vs-vaex-vs-modin-vs-rapids-vs-ray

# Modin

In [ ]:
!pip install modin[all] # Install all of the above

In [1]:
import datetime
from dateutil.parser import parse

In [4]:
import os
import modin.pandas as pd
path='/content/drive/My Drive/Data/Sonion/V04R-V04R-SQLData.dat' # 1.88 gb

## Dask

In [5]:
%%time
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
df_dask = pd.read_csv(path,sep='\t',header=None)

CPU times: user 2.77 s, sys: 2.04 s, total: 4.82 s
Wall time: 1min 22s


In [ ]:
#import pandas as pd
#df = pd.read_csv(f, parse_dates={'Date': [0,1,2]}, index_col='Date')
#df = pd.read_csv("f500_new.csv" , encoding = "iso-8859-1" , 
#parse_dates = {'date_col' : ["day", "month", "year"]} )
#df['date_col'] = pd.to_datetime(df['date_col'])

#import pandas as pd
#mydateparser = lambda x: pd.datetime.strptime(x, "%Y %d %m %H:%M:%S")
#df = pd.read_csv("f500_new.csv" , encoding = "iso-8859-1", parse_dates = ['date_col'] , 
#date_parser = mydateparser)

In [6]:
%%time
df_dask.head()

CPU times: user 2.94 ms, sys: 177 µs, total: 3.12 ms
Wall time: 9.12 ms


,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
0,DEAFAULT,1,33AE015,11,0.0477,12/22/2017 8:44:4,1,0,300,2.34,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
1,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:44:53,1,0,300,2.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
2,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:15,1,0,300,2.33,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
3,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:45:54,1,0,300,2.32,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
4,DEAFAULT,1,33ae015,11,0.0477,12/22/2017 8:46:28,1,0,300,2.36,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


Cả dask và ray đều dùng ram để đọc, sau đó giải phóng ram về ban đầu (mất khoảng gần 2gb ram)

In [7]:
%%time
df_dask[2].value_counts()

CPU times: user 75.2 ms, sys: 21.1 ms, total: 96.3 ms
Wall time: 1.56 s


SU3037_0112       2331513
SU3032_0112        521965
SU3036_0110        208614
CP_SU3032_0112      13088
su3037_0112          7095
su3032_0112          1726
su3036_0110           533
SU3037_0112t          342
SU2010_0112           305
5820                   75
cp_su3032_0112         50
tSU3037_0112           45
33ae015                10
su3037_0112t            3
5070                    3
EST65DB01               2
t-su_ritep01            1
CP_SU3037_0112          1
33AE015                 1
dtype: int64

In [8]:
%%time
df_product_dask=df_dask[df_dask[2]=='CP_SU3032_0112']

CPU times: user 77.2 ms, sys: 30 ms, total: 107 ms
Wall time: 1.47 s


In [9]:
df_product_dask.head()

,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
19604,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117,0.01349,1/16/2018 7:41:45,1,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
19605,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117,0.01349,1/16/2018 7:41:55,1,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
19606,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117,0.01349,1/16/2018 7:42:4,1,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
19607,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117,0.01349,1/16/2018 7:42:13,1,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
19608,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117,0.01349,1/16/2018 7:43:21,1,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


In [10]:
%%time
len(df_product_dask)

CPU times: user 17 µs, sys: 8 µs, total: 25 µs
Wall time: 26.7 µs


13088

In [12]:
%%time
df_product_dask[5]=df_product_dask[5].astype('M8[us]')
df_product_dask[5]

CPU times: user 272 ms, sys: 47.7 ms, total: 319 ms
Wall time: 1.35 s


In [15]:
df_product_dask[5]=df_product_dask[5].dt.date
df_product_dask[5]

19604      2018-01-16
19605      2018-01-16
19606      2018-01-16
19607      2018-01-16
19608      2018-01-16
              ...    
3057035    2019-09-16
3057036    2019-09-16
3057037    2019-09-16
3057038    2019-09-16
3057039    2019-09-16
Name: 5, Length: 13088, dtype: object

In [18]:
df_product_dask

,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
19604,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19605,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19606,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19607,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19608,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057035,C67LSG2,CP_SU30320_0112_60719177 small m2 wk38,CP_SU3032_0112,5820.0,0.01366,2019-09-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3057036,C67LSG2,CP_SU30320_0112_60719177 small m2 wk38,CP_SU3032_0112,5820.0,0.01366,2019-09-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3057037,C67LSG2,CP_SU30320_0112_60719177 small m2 wk38,CP_SU3032_0112,5820.0,0.01366,2019-09-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3057038,C67LSG2,CP_SU30320_0112_60719177 small m2 wk38,CP_SU3032_0112,5820.0,0.01366,2019-09-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [19]:
df_product_dask.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            126, 127, 128, 129, 130, 131, 132, 133, 134, 135],
           dtype='int64', length=136)

In [25]:
df_product_dask.columns = ['c' + str(col)  for col in df_product_dask.columns]

In [26]:
df_product_dask.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c125,c126,c127,c128,c129,c130,c131,c132,c133,c134,c135
19604,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19605,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19606,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19607,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19608,DEFAULT,CP_SU3032_0112_58536396_SMALL_M1_WK3,CP_SU3032_0112,7117.0,0.01349,2018-01-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [27]:
%%time
df_group_date=df_product_dask.groupby('c5')['c59'].mean()
df_group_date

CPU times: user 83.9 ms, sys: 6.91 ms, total: 90.8 ms
Wall time: 189 ms


In [28]:
df_group_date

c5
2018-01-16    108.563685
2018-01-22    108.373110
2018-01-30    108.158194
2018-02-01    108.517000
2018-02-05    108.426425
                 ...    
2019-08-05    108.926915
2019-08-19    108.733868
2019-08-26    109.037876
2019-09-10    108.908286
2019-09-16    108.352898
Name: c59, Length: 107, dtype: float64

In [30]:
%%time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig =go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=df_group_date.index, y=df_group_date.values,
               name="Mean",line=dict( color='#FF5733'),mode='lines+markers')
)

fig.show()

CPU times: user 1.01 s, sys: 258 ms, total: 1.26 s
Wall time: 2.05 s


## Đọc file năng

In [31]:
%%time
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
df = pd.read_csv(path,sep='\t',header=None)

CPU times: user 2.5 s, sys: 1.94 s, total: 4.44 s
Wall time: 1min 4s


In [ ]:
df.columns = ['c' + str(col)  for col in df.columns]

In [33]:
%%time
df_filter=df[df['c2']=='SU3037_0112'] # 2 tr rows
df_filter

CPU times: user 1.52 s, sys: 156 ms, total: 1.68 s
Wall time: 3.8 s


In [34]:
%%time
df_filter.describe()

(pid=295) E0831 09:54:15.029343   295   295 plasma_store_provider.cc:108] Failed to put object 20564e94a042b510ffffffff010000c001000000 in object store because it is full. Object size is 159677317 bytes.
(pid=295) Waiting 1000ms for space to free up...
CPU times: user 2.58 s, sys: 528 ms, total: 3.11 s
Wall time: 1min 21s


,3,6,7,8,9,10,11,12,13,14,15,125,126,127,128,129,130,131,132,133,134,135
count,2.331513e+06,2.331513e+06,2331513.0,2331513.0,2.331513e+06,2331513.0,2331513.0,2.331513e+06,2331513.0,2331513.0,2331513.0,2.331513e+06,2.331513e+06,2331513.0,2331513.0,2331513.0,2331513.0,2331513.0,2331513.0,2331513.0,2331513.0,0.0
mean,9.985860e+03,1.000434e+00,0.0,933.0,3.103666e+00,0.0,1400.0,1.882470e+00,0.0,0.0,0.0,3.327882e-03,1.214233e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
std,6.501723e+03,4.065484e-02,0.0,0.0,2.502048e+00,0.0,0.0,2.186195e+00,0.0,0.0,0.0,1.495133e-01,8.404189e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
min,0.000000e+00,0.000000e+00,0.0,933.0,0.000000e+00,0.0,1400.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
25%,5.070000e+03,1.000000e+00,0.0,933.0,2.440000e+00,0.0,1400.0,1.070000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
50%,7.117000e+03,1.000000e+00,0.0,933.0,2.900000e+00,0.0,1400.0,1.630000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
75%,1.425800e+04,1.000000e+00,0.0,933.0,3.500000e+00,0.0,1400.0,2.370000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
max,6.137010e+05,3.000000e+00,0.0,933.0,1.339840e+03,0.0,1400.0,8.811300e+02,0.0,0.0,0.0,9.000000e+00,9.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [35]:
import datetime
from dateutil.parser import parse

In [38]:
%%time
df_filter['c5']=df_filter['c5'].astype('M8[us]')
df_filter['c5']=df_filter['c5'].dt.date
df_filter['c5']

CPU times: user 50.6 ms, sys: 2.22 ms, total: 52.9 ms
Wall time: 120 ms


In [39]:
%%time
df_group_date=df_filter.groupby('c5')['c65'].mean() # 1 Khz
df_group_date

CPU times: user 11.8 s, sys: 1.54 s, total: 13.4 s
Wall time: 4min 49s


In [40]:
%%time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig =go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=df_group_date.index, y=df_group_date.values,
               name="Mean",line=dict( color='#FF5733'),mode='lines+markers')
)

fig.show()

CPU times: user 206 ms, sys: 0 ns, total: 206 ms
Wall time: 251 ms


In [29]:
#%%time
#meta = ('time', pd.Timestamp)
#df_product_dask[5].map_partitions(pd.to_datetime, meta=meta)

In [ ]:
#my_ddf['time'].map_partitions(to_datetime, columns='time').compute(get=dask.async.get_sync)

In [ ]:
#%time to_datetime(dask_df['time'].compute(), format="%Y%m%d %H:%M:%S.%f" )

#%time to_datetime(pandas_df.time, format="%Y%m%d %H:%M:%S.%f")

In [ ]:
#%%time
#df_product_dask[5]=df_product_dask[5].apply(lambda x: parse(x) )
#df_product_dask[5]=df_product_dask[5].apply(lambda x: x.date() )

## Ray

In [ ]:
%%time
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
df_ray = pd.read_csv(path,sep='\t',header=None)

In [ ]:
%%time
df_ray[2].value_counts()

CPU times: user 60.2 ms, sys: 24 ms, total: 84.2 ms
Wall time: 1.47 s


SU3037_0112       2331513
SU3032_0112        521965
SU3036_0110        208614
CP_SU3032_0112      13088
su3037_0112          7095
su3032_0112          1726
su3036_0110           533
SU3037_0112t          342
SU2010_0112           305
5820                   75
cp_su3032_0112         50
tSU3037_0112           45
33ae015                10
su3037_0112t            3
5070                    3
EST65DB01               2
t-su_ritep01            1
CP_SU3037_0112          1
33AE015                 1
dtype: int64

In [ ]:
%%time
df_product_ray=df_ray[df_ray[2]=='SU3037_0112']

CPU times: user 1.3 s, sys: 93.8 ms, total: 1.39 s
Wall time: 2.94 s


In [ ]:
df_product_ray.head()

,0,1,2,3,4,5,6,7,8,9,10,125,126,127,128,129,130,131,132,133,134,135
11,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:39,1,0,933,2.82,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
12,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:6:53,1,0,933,3.00,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
13,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:1,1,0,933,2.96,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
14,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:18,1,0,933,5.35,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN
15,DEFAULT,58536826 medium m1 wk3,SU3037_0112,7117,0.01349,1/12/2018 14:7:22,1,0,933,3.40,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN


In [ ]:
%%time
len(df_product_ray)

CPU times: user 27 µs, sys: 8 µs, total: 35 µs
Wall time: 39.1 µs


2331513

In [ ]:
%%time
df_product_ray[5]=df_product_ray[5].apply(lambda x: parse(x) )
df_product_ray[5]=df_product_ray[5].apply(lambda x: x.date() )

KeyboardInterrupt: ignored